In [3]:
# pip install spacy python-docx PyPDF2 pandas
# python -m spacy download en_core_web_sm


Using NER and python help with a code that upload a file (doc, csv, pdf, jason) then make nlt analysis (tokenizacion, remove stopword, lematizacion, steaming, etc) finally  return a dict with the name, dates, places. 

con NLP analysis

In [4]:
import nltk
nltk.download(['punkt', 'stopwords', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'words'])


ModuleNotFoundError: No module named 'nltk'

In [3]:
import docx
import pandas as pd
import PyPDF2
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import ne_chunk, pos_tag
from nltk.tree import Tree
###import spacy

# Initialize spaCy for NER (Named Entity Recognition)
##nlp = spacy.load("en_core_web_sm")

# Initialize NLTK tools
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Helper function for reading files
def read_file(file_path):
    ext = file_path.split('.')[-1].lower()
    
    # Read DOCX files
    if ext == 'docx':
        doc = docx.Document(file_path)
        text = '\n'.join([para.text for para in doc.paragraphs])
    # Read CSV files (assuming the text is in the first column)
    elif ext == 'csv':
        df = pd.read_csv(file_path)
        text = ' '.join(df.iloc[:, 0].dropna().astype(str))
    # Read PDF files
    elif ext == 'pdf':
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ''
            for page in reader.pages:
                text += page.extract_text()
    # Read JSON files (assuming JSON has a key 'text')
    elif ext == 'json':
        with open(file_path, 'r') as file:
            data = json.load(file)
            text = data.get('text', '')
    else:
        raise ValueError("Unsupported file type!")
    return text

# Tokenization, stopword removal, lemmatization, stemming
def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Lemmatize and Stem
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    
    return lemmatized_tokens, stemmed_tokens

# Named Entity Recognition (NER)
def extract_entities(text):
    # Use SpaCy NER for extracting names, places, dates, etc.
    doc = nlp(text)
    
    entities = {'names': [], 'places': [], 'dates': []}
    
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            entities['names'].append(ent.text)
        elif ent.label_ == 'GPE':  # Geopolitical Entity (places)
            entities['places'].append(ent.text)
        elif ent.label_ == 'DATE':
            entities['dates'].append(ent.text)
    
    return entities

# Main function to upload file, process text and extract entities
def process_file(file_path):
    # Read the file content
    text = read_file(file_path)
    
    # Preprocess the text
    lemmatized_tokens, stemmed_tokens = preprocess_text(text)
    
    # Extract named entities
    entities = extract_entities(text)
    
    # Create a result dictionary
    result = {
        'file_name': file_path.split('/')[-1],
        'entities': entities,
        'lemmatized_tokens': lemmatized_tokens,
        'stemmed_tokens': stemmed_tokens
    }
    
    return result

# Example usage
file_path = 'Global American Report.pdf'  # Replace with your file path
result = process_file(file_path)
print(result)


NameError: name 'nlp' is not defined